In [1]:
import tensorflow as tf
from simple_model import BasicModel
from robot import DataFlow
import numpy as np

/home/mehdi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# parameters
learning_rate = 0.01
keep_probability = 0.8
unit_size = 128
enc_embed_dim = 256
dec_embed_dim = 256
batch_size = 32
num_layers = 3
data_filename = 'words/eng_finn.p
source_words_filename = 'words/english_words.txt'
target_words_filename = 'words/finn_words.txt'
percent_split_data = 0.9
# model_data = DataFlow(data_filename, source_words_filename, target_words_filename, percent_split_data)

In [3]:
# create the model 
# basic_model = BasicModel(model_data, unit_size, learning_rate, keep_probability, batch_size, num_layers,
#                          enc_embed_dim, dec_embed_dim)
# print('the basic model has been created successfully!')

In [4]:
train_graph = tf.Graph()
with train_graph.as_default():
    model_data = DataFlow(data_filename, source_words_filename, target_words_filename, 
                          percent_split_data, batch_size)
    basic_model = BasicModel(model_data, unit_size, learning_rate, keep_probability, batch_size, num_layers,
                         enc_embed_dim, dec_embed_dim)
    print('the basic model has been created successfully!')
    
    train_data, validation_data = basic_model.prepare_train_validation_data()
    # iterator to switch between the training data and the validation data
    iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
    (source_data, source_lengths), (target_data,  target_lengths) = iterator.get_next()
    # getting the logits and predictions
    logits, predictions = basic_model.model(source_data, 
                                            source_lengths, 
                                            target_data, 
                                            target_lengths, 
                                            mode='train')
    # training phase and getting the loss
    loss, train_op = basic_model.optimizer(logits, target_data, target_lengths)
    print('hello world')

Dataflow object has been created successfully !
the basic model has been created successfully!
 encoder number of tokens :
 num encoder tokens: 15106
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
num decoder tokens: 33039
 we are in the training phase !
congratulations the decoder train phase passed the test !!
we finished the training phase and we reached the inference phase !
congratulations the inference phase passed the test successfully !!!
congratulations ! the training mode is working perfectly !!
hello world


In [ ]:
# training session initialize all the needed inits and variables
train_init = iterator.make_initializer(train_data)
validation_init = iterator.make_initializer(validation_data)
init_variables = tf.global_variables_initializer()
init_tables = tf.tables_initializer()
epochs = 100
train_steps = basic_model._data_flow.nb_train_batches
validation_steps = basic_model._data_flow.nb_val_batches
checkpoints = 'checkpoints/saved_weights.ckpt'



save_path = 'model'
# start the training
with tf.Session(graph=train_graph) as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    train_epochs_losses = []
    validation_epochs = []
    print('start !!')
    for epoch in range(1, epochs + 1):
        sess.run(train_init)
        
        for mini_batch in range(1, train_steps + 1):
            # make the training and get the loss and the predictions of the training data
            train_loss, _, train_predictions = sess.run([loss, train_op, predictions])
            print(train_predictions)
            print('epoch: {} batch: {}/{}\tloss: {}'.format(epoch, batch_size * mini_batch, batch_size * train_steps, train_loss))
            if mini_batch == train_steps:
                print('', end='\t')
            
            
        train_epochs_losses.append(train_loss) # append all the last lossess of all epochs 
        sess.run(validation_init)   # swap to the validation data
        validation_losses = []
        for _ in range(1, validation_steps + 1):
            # get the predictions and losses of the validation data
            loss_validation, predictions_val = sess.run([loss, predictions])
            validation_losses.append(loss_val)
        # calculate the loss mean of all the validation data mini batches    
        validation_loss = np.mean(validation_losses)
        print('validation loss: {:>4.f4}'.format(validation_loss))
        print('--' * 20)
        if epoch % 5 == 0:
            print('saving the brain')
            saver.save(sess, checkpoints)
        
        
        
        
        


start !!
